**Сделать классификацию данных fakenews**

Используя данные fakenews, 3 раза разными способами получить на задаче классификации значение f1 выше 0.91 для методов на sklearn и выше 0.52 для методов на pytorch.

In [ ]:
!wget https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv

--2023-04-09 12:06:47--  https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1253562 (1.2M) [text/plain]
Saving to: ‘Constraint_Train.csv’

Constraint_Train.cs 100%[===================>]   1.20M  --.-KB/s    in 0.01s   

2023-04-09 12:06:47 (114 MB/s) - ‘Constraint_Train.csv’ saved [1253562/1253562]



In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('Constraint_Train.csv')

In [ ]:
df.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [ ]:
from nltk.corpus import stopwords

In [ ]:
from nltk.tokenize import word_tokenize
from string import punctuation

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
! pip install pymorphy2
import pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 38.6 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=0af981fefc434ae1e8ee4750ac123faadcbe658f3ba4a5c83aa5851b09510d4a
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [ ]:
m = pymorphy2.MorphAnalyzer()

In [ ]:
def pre_process(text): # функция для преобразования,очистки текста и лемматизации
  doc_out = ""
  for token in word_tokenize(text):
    if token not in stopwords.words('english') and token not in punctuation:
      doc_out = doc_out + " " + m.parse(token)[0].normal_form
  return doc_out

In [ ]:
df['text_clear'] = df.tweet.apply(pre_process)

In [ ]:
df['text_clear'].head()

0     the cdc currently reports 99031 deaths in gen...
1     states reported 1121 deaths small rise last t...
2     politically correct woman almost uses pandemi...
3     indiafightscorona we 1524 covid testing labor...
4     populous states generate large case counts lo...
Name: text_clear, dtype: object

**CountVectorizer и RandomForestClassifier**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['text_clear'], df.label, test_size=0.2, random_state = 42)

In [ ]:
vec = CountVectorizer()

In [ ]:
bow = vec.fit_transform(X_train)

In [ ]:
rfc = RandomForestClassifier(random_state=42)

In [ ]:
rfc.fit(bow,y_train)

RandomForestClassifier(random_state=42)

In [ ]:
pred_rfc = rfc.predict(vec.transform(X_test))

In [ ]:
print(classification_report(pred_rfc, y_test))

              precision    recall  f1-score   support

        fake       0.93      0.93      0.93       628
        real       0.93      0.93      0.93       656

    accuracy                           0.93      1284
   macro avg       0.93      0.93      0.93      1284
weighted avg       0.93      0.93      0.93      1284



**TfidfVectorizer и Логистическая регрессия**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(max_df = 0.7)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
tfidf_vec = tfidf.fit_transform(X_train)

In [ ]:
model = LogisticRegression(random_state=42)

In [ ]:
model.fit(tfidf_vec, y_train)

LogisticRegression(random_state=42)

In [ ]:
pred= model.predict(tfidf.transform(X_test))

In [ ]:
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

        fake       0.91      0.94      0.93       603
        real       0.95      0.92      0.93       681

    accuracy                           0.93      1284
   macro avg       0.93      0.93      0.93      1284
weighted avg       0.93      0.93      0.93      1284



**Word2Vec и PyTorch**

In [ ]:
from gensim.models.word2vec import Word2Vec

In [ ]:
import numpy as np

In [ ]:
from tqdm import tqdm

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
sentences = [word_tokenize(text.lower()) for text in df.tweet]

In [ ]:
model_tweets = Word2Vec(sentences, workers=4, vector_size=300, min_count=3, window=5)

In [ ]:
model_tweets.init_sims()

<ipython-input-36-f1a8336e4a1e>:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model_tweets.init_sims()


In [ ]:
def get_text_embedding(text):
    result = []
    for word in word_tokenize(text.lower()):
        if word in model_tweets.wv:
            result.append(model_tweets.wv[word])

    if len(result):
        result = np.average(result, axis=0)
    else:
        result = np.zeros(300)
    return result

In [ ]:
features = [get_text_embedding(text) for text in df.tweet]

In [ ]:
labels = (df.label == 'real').astype(int).to_list()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.25)

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.out = nn.Linear(300, 1)

    def forward(self, x):
        return torch.sigmoid(self.out(x))


net = Net()
net.cpu()
print(net)

Net(
  (out): Linear(in_features=300, out_features=1, bias=True)
)


In [ ]:
optimizer = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
in_data = torch.tensor(X_train).float()
targets = torch.tensor(y_train).float()

In [ ]:
def train_one_epoch(in_data, targets, batch_size=16):
    for i in tqdm(range(0, in_data.shape[0], batch_size)):
        batch_x = in_data[i:i + batch_size].cpu()
        batch_y = targets[i:i + batch_size].cpu()
        optimizer.zero_grad()
        output = net(batch_x)
        loss = criterion(output.squeeze(), batch_y)
        loss.backward()
        optimizer.step()
    print(loss)

In [ ]:
for i in range(20):
  train_one_epoch(in_data, targets)

100%|██████████| 301/301 [00:00<00:00, 1015.07it/s]


tensor(0.7167, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


100%|██████████| 301/301 [00:00<00:00, 1586.14it/s]


tensor(0.7037, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


100%|██████████| 301/301 [00:00<00:00, 2099.67it/s]


...

tensor(0.6601, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


100%|██████████| 301/301 [00:00<00:00, 2336.17it/s]


tensor(0.6592, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


100%|██████████| 301/301 [00:00<00:00, 2452.39it/s]

tensor(0.6584, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


In [ ]:
in_data_test = torch.tensor(X_test).float()
targets_test = torch.tensor(y_test).float()

In [ ]:
with torch.no_grad():
    output = net(in_data_test.cpu()).squeeze(1)

In [ ]:
targets_test

tensor([0., 1., 1.,  ..., 0., 1., 0.])

In [ ]:
output.cpu()

tensor([0.1822, 0.7893, 0.3771,  ..., 0.0614, 0.3249, 0.2778])

In [ ]:
pedict = []
for i in output.cpu():
  if i > 0.5:
    res = 1
  else:
    res = 0  
  pedict.append(res)   

In [ ]:
pedict = torch.tensor(pedict)

In [ ]:
print(classification_report(pedict, targets_test))

              precision    recall  f1-score   support

           0       0.93      0.63      0.75      1116
           1       0.51      0.89      0.65       489

    accuracy                           0.71      1605
   macro avg       0.72      0.76      0.70      1605
weighted avg       0.80      0.71      0.72      1605

